In [ ]:
from langchain_ibm import ChatWatsonx
from ibm_watsonx_ai import APIClient
from langchain_core.messages import AIMessage, HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from ibm_watsonx_ai.foundation_models.utils import Tool, Toolkit
import json
import requests

In [ ]:
import os
import getpass

def get_credentials():
	return {
		"url" : "https://us-south.ml.cloud.ibm.com",
		"apikey" : getpass.getpass("Please enter your api key (hit enter): ")
	}

def get_bearer_token():
    url = "https://iam.cloud.ibm.com/identity/token"
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    data = f"grant_type=urn:ibm:params:oauth:grant-type:apikey&apikey={credentials['apikey']}"

    response = requests.post(url, headers=headers, data=data)
    return response.json().get("access_token")

credentials = get_credentials()

In [ ]:
model_id = "ibm/granite-3-3-8b-instruct"

In [ ]:
parameters = {
    "frequency_penalty": 0,
    "max_tokens": 2000,
    "presence_penalty": 0,
    "temperature": 0,
    "top_p": 1
}

In [ ]:
project_id = os.getenv("PROJECT_ID")
space_id = os.getenv("SPACE_ID")


In [ ]:
client = APIClient(credentials=credentials, project_id=project_id, space_id=space_id)

# Create the chat model
def create_chat_model():
    chat_model = ChatWatsonx(
        model_id=model_id,
        url=credentials["url"],
        space_id=space_id,
        project_id=project_id,
        params=parameters,
        watsonx_client=client,
    )
    return chat_model
     

In [ ]:
from ibm_watsonx_ai.deployments import RuntimeContext

context = RuntimeContext(api_client=client)




def create_utility_agent_tool(tool_name, params, api_client, **kwargs):
    from langchain_core.tools import StructuredTool
    utility_agent_tool = Toolkit(
        api_client=api_client
    ).get_tool(tool_name)

    tool_description = utility_agent_tool.get("description")

    if (kwargs.get("tool_description")):
        tool_description = kwargs.get("tool_description")
    elif (utility_agent_tool.get("agent_description")):
        tool_description = utility_agent_tool.get("agent_description")
    
    tool_schema = utility_agent_tool.get("input_schema")
    if (tool_schema == None):
        tool_schema = {
            "type": "object",
            "additionalProperties": False,
            "$schema": "http://json-schema.org/draft-07/schema#",
            "properties": {
                "input": {
                    "description": "input for the tool",
                    "type": "string"
                }
            }
        }
    
    def run_tool(**tool_input):
        query = tool_input
        if (utility_agent_tool.get("input_schema") == None):
            query = tool_input.get("input")

        results = utility_agent_tool.run(
            input=query,
            config=params
        )
        
        return results.get("output")
    
    return StructuredTool(
        name=tool_name,
        description = tool_description,
        func=run_tool,
        args_schema=tool_schema
    )


def create_custom_tool(tool_name, tool_description, tool_code, tool_schema, tool_params):
    from langchain_core.tools import StructuredTool
    import ast

    def call_tool(**kwargs):
        tree = ast.parse(tool_code, mode="exec")
        custom_tool_functions = [ x for x in tree.body if isinstance(x, ast.FunctionDef) ]
        function_name = custom_tool_functions[0].name
        compiled_code = compile(tree, 'custom_tool', 'exec')
        namespace = tool_params if tool_params else {}
        exec(compiled_code, namespace)
        return namespace[function_name](**kwargs)
        
    tool = StructuredTool(
        name=tool_name,
        description = tool_description,
        func=call_tool,
        args_schema=tool_schema
    )
    return tool

def create_custom_tools():
    custom_tools = []


def create_tools(context):
    tools = []
    
    config = {
    }
    tools.append(create_utility_agent_tool("Weather", config, client))
    config = {
    }
    tools.append(create_utility_agent_tool("WebCrawler", config, client))
    config = {
    }
    tools.append(create_utility_agent_tool("DuckDuckGo", config, client))
    config = {
        "maxResults": 5
    }
    tools.append(create_utility_agent_tool("Wikipedia", config, client))
    config = {
        "maxResults": 10
    }
    tools.append(create_utility_agent_tool("GoogleSearch", config, client))

    return tools


In [ ]:
def create_agent(context):
    # Initialize the agent
    chat_model = create_chat_model()
    tools = create_tools(context)

    memory = MemorySaver()
    instructions = """# Notes
- Use markdown syntax for formatting code snippets, links, JSON, tables, images, files.
- Any HTML tags must be wrapped in block quotes, for example ``````.
- When returning code blocks, specify language.
- Sometimes, things don't go as planned. Tools may not provide useful information on the first few tries. You should always try a few different approaches before declaring the problem unsolvable.
- When the tool doesn't give you what you were asking for, you must either use another tool or a different tool input.
- When using search engines, you try different formulations of the query, possibly even in a different language.
- You cannot do complex calculations, computations, or data manipulations without using tools.
- If you need to call a tool to compute something, always call it instead of saying you will call it.

If a tool returns an IMAGE in the result, you must include it in your answer as Markdown.

Example:

Tool result: IMAGE({commonApiUrl}/wx/v1-beta/utility_agent_tools/cache/images/plt-04e3c91ae04b47f8934a4e6b7d1fdc2c.png)
Markdown to return to user: ![Generated image]({commonApiUrl}/wx/v1-beta/utility_agent_tools/cache/images/plt-04e3c91ae04b47f8934a4e6b7d1fdc2c.png)


# Notes - Use markdown syntax for formatting code snippets, links, JSON, tables, images, files. - Any HTML tags must be wrapped in block quotes, for example ``````. - When returning code blocks, specify language. - Sometimes, things don't go as planned. Tools may not provide useful information on the first few tries. You should always try a few different approaches before declaring the problem unsolvable. - When the tool doesn't give you what you were asking for, you must either use another tool or a different tool input. - When using search engines, you try different formulations of the query, possibly even in a different language. - You cannot do complex calculations, computations, or data manipulations without using tools. - If you need to call a tool to compute something, always call it instead of saying you will call it. If a tool returns an IMAGE in the result, you must include it in your answer as Markdown. Example: Tool result: IMAGE({commonApiUrl}/wx/v1-beta/utility_agent_tools/cache/images/plt-04e3c91ae04b47f8934a4e6b7d1fdc2c.png) Markdown to return to user: ![Generated image]({commonApiUrl}/wx/v1-beta/utility_agent_tools/cache/images/plt-04e3c91ae04b47f8934a4e6b7d1fdc2c.png) 


You are a smart, friendly, and helpful AI assistant.

🎯 Your role:
- Assist users by answering their questions clearly and politely.
- Always stay helpful, informative, and respectful.
- Think step-by-step when the task involves actions, instructions, or decision-making.

🧠 Response Guidelines:
- Keep answers short, simple, and beginner-friendly.
- Use bullet points or numbered steps for clarity.
- Avoid over-complicating or using technical jargon (unless necessary and explained).
- Use examples wherever possible to help users relate.

🔐 Safety:
- Do not collect, store, or request any personal information (like Aadhaar, phone numbers, passwords, OTPs, or bank details).
- If the user shares sensitive info, gently inform them it's not needed for your help and advise caution.

🌐 Language & Tone:
- Be polite, inclusive, and neutral.
- Use a warm and supportive tone (not robotic).
- Support multiple languages if instructed (e.g., Hindi, Telugu).
- Adapt answers to local/cultural context when needed.

📌 If you don’t have a direct answer:
- Say something like: “Based on the information I have…” or “I recommend checking this official link for the most accurate details.”
- Never say “I don’t know” bluntly.

✅ Always aim to:
- Educate the user
- Protect the user from risks
- Build trust and confidence in digital tools

You are a helpful assistant that uses tools to answer questions in detail.
When greeted, say \"Hi, I am FinAdvisor ai agent. How can I help you?\"

🧠 You are a helpful AI assistant trained to improve digital financial literacy for citizens, especially beginners, rural users, and the elderly.

🎯 Your job is to:
- Answer questions about UPI, net banking, loans, scams, budgeting, and government financial schemes.
- Explain things simply, like a friendly teacher.
- Avoid technical jargon unless explained clearly.
- Never collect or ask for personal information like Aadhaar, OTP, or bank details.

🔐 Safety first:
- Always warn users about phishing, scams, and OTP misuse.
- Suggest reporting fraud to https://cybercrime.gov.in or contacting their bank.

🌐 Use trusted sources only:
- RBI official website
- NPCI (https://www.npci.org.in)
- Government financial portals like Jan Dhan, NSAP, etc.

🗣️ Multilingual support:
- Be ready to assist in English and regional languages (e.g., Hindi, Telugu) if asked.
- Keep tone respectful and culturally sensitive.
"""

    agent = create_react_agent(chat_model, tools=tools, checkpointer=memory, state_modifier=instructions)

    return agent
     

# Visualize the graph
from IPython.display import Image, display
from langchain_core.runnables.graph import CurveStyle, MermaidDrawMethod, NodeStyles

Image(
    create_agent(context).get_graph().draw_mermaid_png(
        draw_method=MermaidDrawMethod.API,
    )
)    


In [ ]:
agent = create_agent(context)

def convert_messages(messages):
    converted_messages = []
    for message in messages:
        if (message["role"] == "user"):
            converted_messages.append(HumanMessage(content=message["content"]))
        elif (message["role"] == "assistant"):
            converted_messages.append(AIMessage(content=message["content"]))
    return converted_messages

question = input("Question: ")

messages = [{
    "role": "user",
    "content": question
}]

generated_response = agent.invoke(
    { "messages": convert_messages(messages) },
    { "configurable": { "thread_id": "42" } }
)

print_full_response = False

if (print_full_response):
    print(generated_response)
else:
    result = generated_response["messages"][-1].content
    print(f"Agent: {result}")